# Crop Price Prediction Module
Descriptive market analysis plus a simple linear regression forecast.

In [ ]:
import os
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
from PIL import Image
from sklearn.linear_model import LinearRegression
from IPython.display import display

In [ ]:
def find_dataset_root():
    candidates = [
        os.path.join(os.getcwd(), "dataset"),
        os.path.join(os.path.dirname(os.getcwd()), "dataset")
    ]
    for path in candidates:
        if os.path.isdir(path):
            return path
    raise FileNotFoundError("Dataset folder not found. Expected ./dataset or ../dataset")

DATASET_ROOT = find_dataset_root()
DATASET_PATH = os.path.join(
    DATASET_ROOT,
    "anshtanwar",
    "current-daily-price-of-various-commodities-india",
    "Price_Agriculture_commodities_Week.csv"
)

if not os.path.exists(DATASET_PATH):
    raise FileNotFoundError(f"Missing dataset file: {DATASET_PATH}")

market_price_df = pd.read_csv(DATASET_PATH)
market_price_df['Arrival_Date'] = pd.to_datetime(
    market_price_df['Arrival_Date'], errors='coerce', dayfirst=True
)
market_price_df['year'] = market_price_df['Arrival_Date'].dt.year
market_price_df['month'] = market_price_df['Arrival_Date'].dt.month
market_price_df['day'] = market_price_df['Arrival_Date'].dt.day

for price_col in ['Min Price', 'Max Price', 'Modal Price']:
    market_price_df[price_col] = pd.to_numeric(
        market_price_df[price_col], errors='coerce'
    )

if market_price_df.empty:
    raise ValueError('Market price dataset is empty after loading')

print(f"Loaded market prices: {market_price_df.shape}")

In [ ]:
def analyze_market_prices(commodity, state, date_range=30):
    filtered_df = market_price_df[
        & (market_price_df['State'].str.contains(state, case=False, na=False))